In [8]:
import glob
import pandas as pd
import json
import requests

In [9]:
df = pd.concat([pd.read_csv(f) for f in glob.glob("./files/*.csv")], ignore_index=True)
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,NaN
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,NaN
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,NaN
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,NaN
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,NaN


In [10]:
df.shape

(914260, 11)

In [11]:
df.isnull().sum()

month                       0
town                        0
flat_type                   0
block                       0
street_name                 0
storey_range                0
floor_area_sqm              0
flat_model                  0
lease_commence_date         0
resale_price                0
remaining_lease        709050
dtype: int64

In [12]:
# check the number of streets
street_nos = len(df['street_name'].unique())
street_nos

584

In [13]:
# list unique street names
streets = df['street_name'].unique()
streets

array(['ANG MO KIO AVE 1', 'ANG MO KIO AVE 3', 'ANG MO KIO AVE 4',
       'ANG MO KIO AVE 10', 'ANG MO KIO AVE 5', 'ANG MO KIO AVE 8',
       'ANG MO KIO AVE 6', 'ANG MO KIO AVE 9', 'ANG MO KIO AVE 2',
       'BEDOK RESERVOIR RD', 'BEDOK NTH ST 3', 'BEDOK STH RD',
       'NEW UPP CHANGI RD', 'BEDOK NTH RD', 'BEDOK STH AVE 1',
       'CHAI CHEE RD', 'CHAI CHEE DR', 'BEDOK NTH AVE 4',
       'BEDOK STH AVE 3', 'BEDOK STH AVE 2', 'BEDOK NTH ST 2',
       'BEDOK NTH ST 4', 'BEDOK NTH AVE 2', 'BEDOK NTH AVE 3',
       'BEDOK NTH AVE 1', 'BEDOK NTH ST 1', 'CHAI CHEE ST', 'SIN MING RD',
       'SHUNFU RD', 'BT BATOK ST 11', 'BT BATOK WEST AVE 8',
       'BT BATOK WEST AVE 6', 'BT BATOK ST 21', 'BT BATOK EAST AVE 5',
       'BT BATOK EAST AVE 4', 'HILLVIEW AVE', 'BT BATOK CTRL',
       'BT BATOK ST 31', 'BT BATOK EAST AVE 3', 'TAMAN HO SWEE',
       'TELOK BLANGAH CRES', 'BEO CRES', 'TELOK BLANGAH DR', 'DEPOT RD',
       'TELOK BLANGAH RISE', 'JLN BT MERAH', 'HENDERSON RD', 'INDUS RD',
       

In [14]:
# map the streets to povide input to ML model
streets_mapping = {street: idx + 1 for idx, street in enumerate(streets)}
streets_mapping

{'ANG MO KIO AVE 1': 1,
 'ANG MO KIO AVE 3': 2,
 'ANG MO KIO AVE 4': 3,
 'ANG MO KIO AVE 10': 4,
 'ANG MO KIO AVE 5': 5,
 'ANG MO KIO AVE 8': 6,
 'ANG MO KIO AVE 6': 7,
 'ANG MO KIO AVE 9': 8,
 'ANG MO KIO AVE 2': 9,
 'BEDOK RESERVOIR RD': 10,
 'BEDOK NTH ST 3': 11,
 'BEDOK STH RD': 12,
 'NEW UPP CHANGI RD': 13,
 'BEDOK NTH RD': 14,
 'BEDOK STH AVE 1': 15,
 'CHAI CHEE RD': 16,
 'CHAI CHEE DR': 17,
 'BEDOK NTH AVE 4': 18,
 'BEDOK STH AVE 3': 19,
 'BEDOK STH AVE 2': 20,
 'BEDOK NTH ST 2': 21,
 'BEDOK NTH ST 4': 22,
 'BEDOK NTH AVE 2': 23,
 'BEDOK NTH AVE 3': 24,
 'BEDOK NTH AVE 1': 25,
 'BEDOK NTH ST 1': 26,
 'CHAI CHEE ST': 27,
 'SIN MING RD': 28,
 'SHUNFU RD': 29,
 'BT BATOK ST 11': 30,
 'BT BATOK WEST AVE 8': 31,
 'BT BATOK WEST AVE 6': 32,
 'BT BATOK ST 21': 33,
 'BT BATOK EAST AVE 5': 34,
 'BT BATOK EAST AVE 4': 35,
 'HILLVIEW AVE': 36,
 'BT BATOK CTRL': 37,
 'BT BATOK ST 31': 38,
 'BT BATOK EAST AVE 3': 39,
 'TAMAN HO SWEE': 40,
 'TELOK BLANGAH CRES': 41,
 'BEO CRES': 42,
 'TELOK B

In [15]:
# alter the column with the mapped numbers
df['street_name'] = df['street_name'].map(streets_mapping)
df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,1990-01,ANG MO KIO,1 ROOM,309,1,10 TO 12,31.0,IMPROVED,1977,9000.0,NaN
1,1990-01,ANG MO KIO,1 ROOM,309,1,04 TO 06,31.0,IMPROVED,1977,6000.0,NaN
2,1990-01,ANG MO KIO,1 ROOM,309,1,10 TO 12,31.0,IMPROVED,1977,8000.0,NaN
3,1990-01,ANG MO KIO,1 ROOM,309,1,07 TO 09,31.0,IMPROVED,1977,6000.0,NaN
4,1990-01,ANG MO KIO,3 ROOM,216,1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
914255,2014-12,YISHUN,5 ROOM,816,305,10 TO 12,122.0,Improved,1988,580000.0,NaN
914256,2014-12,YISHUN,EXECUTIVE,325,330,10 TO 12,146.0,Maisonette,1988,540000.0,NaN
914257,2014-12,YISHUN,EXECUTIVE,618,219,07 TO 09,164.0,Apartment,1992,738000.0,NaN
914258,2014-12,YISHUN,EXECUTIVE,277,224,07 TO 09,152.0,Maisonette,1985,592000.0,NaN


In [16]:
# function to convert the years and months to years in decimal
def convert_to_years(row):
    if pd.notna(row):
        if isinstance(row, str):
            if 'years' in row:
                parts = row.split()
                years = float(parts[0])
                if len(parts) > 2:  # If there's a month part as well
                    months = float(parts[2])
                    return years + months / 12
                else:
                    return years
        else:
          return row
    return row

In [17]:
# Apply the conversion function to the DataFrame column
df['remaining_lease'] = df['remaining_lease'].apply(convert_to_years)
df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,1990-01,ANG MO KIO,1 ROOM,309,1,10 TO 12,31.0,IMPROVED,1977,9000.0,NaN
1,1990-01,ANG MO KIO,1 ROOM,309,1,04 TO 06,31.0,IMPROVED,1977,6000.0,NaN
2,1990-01,ANG MO KIO,1 ROOM,309,1,10 TO 12,31.0,IMPROVED,1977,8000.0,NaN
3,1990-01,ANG MO KIO,1 ROOM,309,1,07 TO 09,31.0,IMPROVED,1977,6000.0,NaN
4,1990-01,ANG MO KIO,3 ROOM,216,1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
914255,2014-12,YISHUN,5 ROOM,816,305,10 TO 12,122.0,Improved,1988,580000.0,NaN
914256,2014-12,YISHUN,EXECUTIVE,325,330,10 TO 12,146.0,Maisonette,1988,540000.0,NaN
914257,2014-12,YISHUN,EXECUTIVE,618,219,07 TO 09,164.0,Apartment,1992,738000.0,NaN
914258,2014-12,YISHUN,EXECUTIVE,277,224,07 TO 09,152.0,Maisonette,1985,592000.0,NaN


In [18]:
# fill null values in remaining_lease with its mode
df['remaining_lease']= df['remaining_lease'].fillna(df['remaining_lease'].mode()[0])
df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,1990-01,ANG MO KIO,1 ROOM,309,1,10 TO 12,31.0,IMPROVED,1977,9000.0,68.0
1,1990-01,ANG MO KIO,1 ROOM,309,1,04 TO 06,31.0,IMPROVED,1977,6000.0,68.0
2,1990-01,ANG MO KIO,1 ROOM,309,1,10 TO 12,31.0,IMPROVED,1977,8000.0,68.0
3,1990-01,ANG MO KIO,1 ROOM,309,1,07 TO 09,31.0,IMPROVED,1977,6000.0,68.0
4,1990-01,ANG MO KIO,3 ROOM,216,1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,68.0
...,...,...,...,...,...,...,...,...,...,...,...
914255,2014-12,YISHUN,5 ROOM,816,305,10 TO 12,122.0,Improved,1988,580000.0,68.0
914256,2014-12,YISHUN,EXECUTIVE,325,330,10 TO 12,146.0,Maisonette,1988,540000.0,68.0
914257,2014-12,YISHUN,EXECUTIVE,618,219,07 TO 09,164.0,Apartment,1992,738000.0,68.0
914258,2014-12,YISHUN,EXECUTIVE,277,224,07 TO 09,152.0,Maisonette,1985,592000.0,68.0


In [19]:
# Split the 'Range' column into lower and upper bounds
df[['storey_lower_bound', 'storey_upper_bound']] = df['storey_range'].str.split(' TO ', expand=True)
# Convert the lower and upper bounds to numeric values
df['storey_lower_bound'] = pd.to_numeric(df['storey_lower_bound'])
df['storey_upper_bound'] = pd.to_numeric(df['storey_upper_bound'])
df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,storey_lower_bound,storey_upper_bound
0,1990-01,ANG MO KIO,1 ROOM,309,1,10 TO 12,31.0,IMPROVED,1977,9000.0,68.0,10,12
1,1990-01,ANG MO KIO,1 ROOM,309,1,04 TO 06,31.0,IMPROVED,1977,6000.0,68.0,4,6
2,1990-01,ANG MO KIO,1 ROOM,309,1,10 TO 12,31.0,IMPROVED,1977,8000.0,68.0,10,12
3,1990-01,ANG MO KIO,1 ROOM,309,1,07 TO 09,31.0,IMPROVED,1977,6000.0,68.0,7,9
4,1990-01,ANG MO KIO,3 ROOM,216,1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,68.0,4,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
914255,2014-12,YISHUN,5 ROOM,816,305,10 TO 12,122.0,Improved,1988,580000.0,68.0,10,12
914256,2014-12,YISHUN,EXECUTIVE,325,330,10 TO 12,146.0,Maisonette,1988,540000.0,68.0,10,12
914257,2014-12,YISHUN,EXECUTIVE,618,219,07 TO 09,164.0,Apartment,1992,738000.0,68.0,7,9
914258,2014-12,YISHUN,EXECUTIVE,277,224,07 TO 09,152.0,Maisonette,1985,592000.0,68.0,7,9


In [20]:
# drop storey range column
df.drop('storey_range', axis=1, inplace=True)
df

,month,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,storey_lower_bound,storey_upper_bound
0,1990-01,ANG MO KIO,1 ROOM,309,1,31.0,IMPROVED,1977,9000.0,68.0,10,12
1,1990-01,ANG MO KIO,1 ROOM,309,1,31.0,IMPROVED,1977,6000.0,68.0,4,6
2,1990-01,ANG MO KIO,1 ROOM,309,1,31.0,IMPROVED,1977,8000.0,68.0,10,12
3,1990-01,ANG MO KIO,1 ROOM,309,1,31.0,IMPROVED,1977,6000.0,68.0,7,9
4,1990-01,ANG MO KIO,3 ROOM,216,1,73.0,NEW GENERATION,1976,47200.0,68.0,4,6
...,...,...,...,...,...,...,...,...,...,...,...,...
914255,2014-12,YISHUN,5 ROOM,816,305,122.0,Improved,1988,580000.0,68.0,10,12
914256,2014-12,YISHUN,EXECUTIVE,325,330,146.0,Maisonette,1988,540000.0,68.0,10,12
914257,2014-12,YISHUN,EXECUTIVE,618,219,164.0,Apartment,1992,738000.0,68.0,7,9
914258,2014-12,YISHUN,EXECUTIVE,277,224,152.0,Maisonette,1985,592000.0,68.0,7,9


In [21]:
# list unique flat type
df['flat_type'].unique()

array(['1 ROOM', '3 ROOM', '4 ROOM', '5 ROOM', '2 ROOM', 'EXECUTIVE',
       'MULTI GENERATION', 'MULTI-GENERATION'], dtype=object)

In [22]:
# Replace 'MULTI-GENERATION' with 'MULTI GENERATION' in the specified column to avoid duplicate
df['flat_type'] = df['flat_type'].str.replace('MULTI-GENERATION', 'MULTI GENERATION')

In [23]:
# list unique flat type
df['flat_type'].unique()

array(['1 ROOM', '3 ROOM', '4 ROOM', '5 ROOM', '2 ROOM', 'EXECUTIVE',
       'MULTI GENERATION'], dtype=object)

In [24]:
# Define a mapping of flat_type to numbers
category_mapping = {
    '1 ROOM': 1,
    '2 ROOM': 2,
    '3 ROOM': 3,
    '4 ROOM': 4,
    '5 ROOM': 5,
    'EXECUTIVE': 6,
    'MULTI GENERATION': 7
}

# alter the column with the mapped numbers
df['flat_type'] = df['flat_type'].map(category_mapping)

# Display the DataFrame
df

,month,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,storey_lower_bound,storey_upper_bound
0,1990-01,ANG MO KIO,1,309,1,31.0,IMPROVED,1977,9000.0,68.0,10,12
1,1990-01,ANG MO KIO,1,309,1,31.0,IMPROVED,1977,6000.0,68.0,4,6
2,1990-01,ANG MO KIO,1,309,1,31.0,IMPROVED,1977,8000.0,68.0,10,12
3,1990-01,ANG MO KIO,1,309,1,31.0,IMPROVED,1977,6000.0,68.0,7,9
4,1990-01,ANG MO KIO,3,216,1,73.0,NEW GENERATION,1976,47200.0,68.0,4,6
...,...,...,...,...,...,...,...,...,...,...,...,...
914255,2014-12,YISHUN,5,816,305,122.0,Improved,1988,580000.0,68.0,10,12
914256,2014-12,YISHUN,6,325,330,146.0,Maisonette,1988,540000.0,68.0,10,12
914257,2014-12,YISHUN,6,618,219,164.0,Apartment,1992,738000.0,68.0,7,9
914258,2014-12,YISHUN,6,277,224,152.0,Maisonette,1985,592000.0,68.0,7,9


In [25]:
# list unique flat model
df['flat_model'].unique()

array(['IMPROVED', 'NEW GENERATION', 'MODEL A', 'STANDARD', 'SIMPLIFIED',
       'MODEL A-MAISONETTE', 'APARTMENT', 'MAISONETTE', 'TERRACE',
       '2-ROOM', 'IMPROVED-MAISONETTE', 'MULTI GENERATION',
       'PREMIUM APARTMENT', 'Improved', 'New Generation', 'Model A',
       'Standard', 'Apartment', 'Simplified', 'Model A-Maisonette',
       'Maisonette', 'Multi Generation', 'Adjoined flat',
       'Premium Apartment', 'Terrace', 'Improved-Maisonette',
       'Premium Maisonette', '2-room', 'Model A2', 'Type S1', 'Type S2',
       'DBSS', 'Premium Apartment Loft', '3Gen'], dtype=object)

In [26]:
# Create a mapping of 'flat_model' values to numbers
flat_model_mapping = {
    'IMPROVED': 1,
    'NEW GENERATION': 2,
    'MODEL A': 3,
    'STANDARD': 4,
    'SIMPLIFIED': 5,
    'MODEL A-MAISONETTE': 6,
    'APARTMENT': 7,
    'MAISONETTE': 8,
    'TERRACE': 9,
    '2-ROOM': 10,
    'IMPROVED-MAISONETTE': 11,
    'MULTI GENERATION': 12,
    'PREMIUM APARTMENT': 13,
    'Improved': 14,
    'New Generation': 15,
    'Model A': 16,
    'Standard': 17,
    'Apartment': 18,
    'Simplified': 19,
    'Model A-Maisonette': 20,
    'Maisonette': 21,
    'Multi Generation': 22,
    'Adjoined flat': 23,
    'Premium Apartment': 24,
    'Terrace': 25,
    'Improved-Maisonette': 26,
    'Premium Maisonette': 27,
    '2-room': 28,
    'Model A2': 29,
    'DBSS': 30,
    'Type S1': 31,
    'Type S2': 32,
    'Premium Apartment Loft': 33,
    '3Gen': 34
}

# Map 'flat_model' values to numbers
df['flat_model'] = df['flat_model'].map(flat_model_mapping)

# Display the DataFrame
df

,month,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,storey_lower_bound,storey_upper_bound
0,1990-01,ANG MO KIO,1,309,1,31.0,1,1977,9000.0,68.0,10,12
1,1990-01,ANG MO KIO,1,309,1,31.0,1,1977,6000.0,68.0,4,6
2,1990-01,ANG MO KIO,1,309,1,31.0,1,1977,8000.0,68.0,10,12
3,1990-01,ANG MO KIO,1,309,1,31.0,1,1977,6000.0,68.0,7,9
4,1990-01,ANG MO KIO,3,216,1,73.0,2,1976,47200.0,68.0,4,6
...,...,...,...,...,...,...,...,...,...,...,...,...
914255,2014-12,YISHUN,5,816,305,122.0,14,1988,580000.0,68.0,10,12
914256,2014-12,YISHUN,6,325,330,146.0,21,1988,540000.0,68.0,10,12
914257,2014-12,YISHUN,6,618,219,164.0,18,1992,738000.0,68.0,7,9
914258,2014-12,YISHUN,6,277,224,152.0,21,1985,592000.0,68.0,7,9


In [28]:
# list unique town names
df['town'].unique()

array(['ANG MO KIO', 'BEDOK', 'BISHAN', 'BUKIT BATOK', 'BUKIT MERAH',
       'BUKIT TIMAH', 'CENTRAL AREA', 'CHOA CHU KANG', 'CLEMENTI',
       'GEYLANG', 'HOUGANG', 'JURONG EAST', 'JURONG WEST',
       'KALLANG/WHAMPOA', 'MARINE PARADE', 'QUEENSTOWN', 'SENGKANG',
       'SERANGOON', 'TAMPINES', 'TOA PAYOH', 'WOODLANDS', 'YISHUN',
       'LIM CHU KANG', 'SEMBAWANG', 'BUKIT PANJANG', 'PASIR RIS',
       'PUNGGOL'], dtype=object)

In [29]:
# Create a mapping of 'town' values to numbers
town_mapping = {
    'ANG MO KIO': 1,
    'BEDOK': 2,
    'BISHAN': 3,
    'BUKIT BATOK': 4,
    'BUKIT MERAH': 5,
    'BUKIT TIMAH': 6,
    'CENTRAL AREA': 7,
    'CHOA CHU KANG': 8,
    'CLEMENTI': 9,
    'GEYLANG': 10,
    'HOUGANG': 11,
    'JURONG EAST': 12,
    'JURONG WEST': 13,
    'KALLANG/WHAMPOA': 14,
    'MARINE PARADE': 15,
    'QUEENSTOWN': 16,
    'SENGKANG': 17,
    'SERANGOON': 18,
    'TAMPINES': 19,
    'TOA PAYOH': 20,
    'WOODLANDS': 21,
    'YISHUN': 22,
    'LIM CHU KANG': 23,
    'SEMBAWANG': 24,
    'BUKIT PANJANG': 25,
    'PASIR RIS': 26,
    'PUNGGOL': 27
}

# Map 'town' values to numbers
df['town'] = df['town'].map(town_mapping)

# Display the DataFrame
df

,month,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,storey_lower_bound,storey_upper_bound
0,1990-01,1,1,309,1,31.0,1,1977,9000.0,68.0,10,12
1,1990-01,1,1,309,1,31.0,1,1977,6000.0,68.0,4,6
2,1990-01,1,1,309,1,31.0,1,1977,8000.0,68.0,10,12
3,1990-01,1,1,309,1,31.0,1,1977,6000.0,68.0,7,9
4,1990-01,1,3,216,1,73.0,2,1976,47200.0,68.0,4,6
...,...,...,...,...,...,...,...,...,...,...,...,...
914255,2014-12,22,5,816,305,122.0,14,1988,580000.0,68.0,10,12
914256,2014-12,22,6,325,330,146.0,21,1988,540000.0,68.0,10,12
914257,2014-12,22,6,618,219,164.0,18,1992,738000.0,68.0,7,9
914258,2014-12,22,6,277,224,152.0,21,1985,592000.0,68.0,7,9


In [30]:
# Define a mapping for letters to decimal values
letter_mapping = {chr(ord('A') + i): f'.{i+1}' for i in range(26)}

# Replace letters with their corresponding decimal values
for letter, decimal in letter_mapping.items():
    df['block'] = df['block'].str.replace(letter, decimal)

# Use pd.to_numeric to convert the column to float and set invalid parsing to NaN
df['block'] = pd.to_numeric(df['block'])

# Display the DataFrame with the column as float values
df

,month,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,storey_lower_bound,storey_upper_bound
0,1990-01,1,1,309.0,1,31.0,1,1977,9000.0,68.0,10,12
1,1990-01,1,1,309.0,1,31.0,1,1977,6000.0,68.0,4,6
2,1990-01,1,1,309.0,1,31.0,1,1977,8000.0,68.0,10,12
3,1990-01,1,1,309.0,1,31.0,1,1977,6000.0,68.0,7,9
4,1990-01,1,3,216.0,1,73.0,2,1976,47200.0,68.0,4,6
...,...,...,...,...,...,...,...,...,...,...,...,...
914255,2014-12,22,5,816.0,305,122.0,14,1988,580000.0,68.0,10,12
914256,2014-12,22,6,325.0,330,146.0,21,1988,540000.0,68.0,10,12
914257,2014-12,22,6,618.0,219,164.0,18,1992,738000.0,68.0,7,9
914258,2014-12,22,6,277.0,224,152.0,21,1985,592000.0,68.0,7,9


In [31]:
# check datatype of the lease resale month column
month = df['month'].dtype
month

dtype('O')

In [32]:
# Separate the year and month of resale
df[['resale_year', 'resale_month']] = df['month'].str.split('-', expand=True)
df

,month,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,storey_lower_bound,storey_upper_bound,resale_year,resale_month
0,1990-01,1,1,309.0,1,31.0,1,1977,9000.0,68.0,10,12,1990,01
1,1990-01,1,1,309.0,1,31.0,1,1977,6000.0,68.0,4,6,1990,01
2,1990-01,1,1,309.0,1,31.0,1,1977,8000.0,68.0,10,12,1990,01
3,1990-01,1,1,309.0,1,31.0,1,1977,6000.0,68.0,7,9,1990,01
4,1990-01,1,3,216.0,1,73.0,2,1976,47200.0,68.0,4,6,1990,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
914255,2014-12,22,5,816.0,305,122.0,14,1988,580000.0,68.0,10,12,2014,12
914256,2014-12,22,6,325.0,330,146.0,21,1988,540000.0,68.0,10,12,2014,12
914257,2014-12,22,6,618.0,219,164.0,18,1992,738000.0,68.0,7,9,2014,12
914258,2014-12,22,6,277.0,224,152.0,21,1985,592000.0,68.0,7,9,2014,12


In [33]:
# drop the resale month column
df.drop('month', axis = 1, inplace= True)
df

,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,storey_lower_bound,storey_upper_bound,resale_year,resale_month
0,1,1,309.0,1,31.0,1,1977,9000.0,68.0,10,12,1990,01
1,1,1,309.0,1,31.0,1,1977,6000.0,68.0,4,6,1990,01
2,1,1,309.0,1,31.0,1,1977,8000.0,68.0,10,12,1990,01
3,1,1,309.0,1,31.0,1,1977,6000.0,68.0,7,9,1990,01
4,1,3,216.0,1,73.0,2,1976,47200.0,68.0,4,6,1990,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
914255,22,5,816.0,305,122.0,14,1988,580000.0,68.0,10,12,2014,12
914256,22,6,325.0,330,146.0,21,1988,540000.0,68.0,10,12,2014,12
914257,22,6,618.0,219,164.0,18,1992,738000.0,68.0,7,9,2014,12
914258,22,6,277.0,224,152.0,21,1985,592000.0,68.0,7,9,2014,12


In [34]:
# check for null values
df.isnull().sum()

town                   0
flat_type              0
block                  0
street_name            0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
resale_price           0
remaining_lease        0
storey_lower_bound     0
storey_upper_bound     0
resale_year            0
resale_month           0
dtype: int64

In [35]:
# import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [36]:
# fetch sample data
c_df = df.sample(n=100000, random_state=42)

In [37]:
# assign feature and target variable
X = c_df.drop('resale_price', axis= 1)
y = c_df['resale_price']

In [38]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [39]:
# Create and train the Random Forest Regressor model
rf_regressor = RandomForestRegressor(random_state=42)
rf_regressor.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [40]:
# Make predictions on the test set
y_pred = rf_regressor.predict(X_test)

In [41]:
# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)

In [42]:
# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)

In [43]:
# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)

In [44]:
# Calculate R-squared (R2) Score
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE)    :", mae)
print("Mean Squared Error (MSE)     :", mse)
print("Root Mean Square Error (RMSE):", rmse)
print("R-squared (R2) Score         :", r2)

Mean Absolute Error (MAE)    : 19081.364264084685
Mean Squared Error (MSE)     : 870011984.8016299
Root Mean Square Error (RMSE): 29495.96556822017
R-squared (R2) Score         : 0.96876743276289


In [45]:
# get sample to predict the resale price
pre_df = df.sample(n=2)
pre_df

,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,storey_lower_bound,storey_upper_bound,resale_year,resale_month
720812,25,4,523.0,431,103.0,16,1998,342000.0,79.5,7,9,2018,05
147502,9,3,371.0,88,68.0,2,1981,207000.0,68.0,10,12,1997,01


In [46]:
# drop the resale price column
A = pre_df.drop('resale_price', axis = 1)

In [49]:
# Make predictions on the sample
pre_pred = rf_regressor.predict(A)
pre_pred

array([362449.76, 203209.  ])

In [48]:
# save the processed data for the deployment
df.to_csv('c.csv')